# 

### 1. 导入必要库并读取数据

In [1]:
from ucimlrepo import fetch_ucirepo 
import torch
import numpy as np

实际上我们可以直接用fetch_ucirepo库来获取数据，上网站自己查询id即可

In [2]:
# fetch dataset 
iris = fetch_ucirepo(id=53) 

# data (as pandas dataframes) 
X = iris.data.features 
y = iris.data.targets 

X = np.array(X)
y = np.array(y)
y = y.reshape(-1)
X,y

(array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
        [5

### 2. 将数据转化为张量

In [3]:
for i in range(y.shape[0]):
    if y[i] == 'Iris-setosa':
        y[i] = 0
    elif y[i] == 'Iris-versicolor':
        y[i] = 1
    else:
        y[i] = 2

# 将 y 转换为整数类型
y = y.astype(np.int64)

# 将 y 转换为 PyTorch 张量
X = torch.tensor(X,dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)  # 使用 long 类型表示整数标签

### 3. 指定device,lr,loss_fn,optimizer和网络模型

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() 
                      else 'mps' if torch.mps.is_available()
                      else 'cpu')
loss_fn = torch.nn.CrossEntropyLoss()

class Classifier(torch.nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.fc1 = torch.nn.Linear(4, 8)
        self.fc2 = torch.nn.Linear(8, 3)
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
model = Classifier().to(device)

optimizer = torch.optim.Adam(model.parameters(),lr=0.1)

### 4. 训练模型

In [5]:
epochs = 1000

for epoch in range(epochs):
    X = X.to(device)
    y = y.to(device)
    model.train()
    y_pred_logit = model(X)
    loss = loss_fn(y_pred_logit, y)
    y_pred_prob = torch.softmax(y_pred_logit,dim=1)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    accuracy = (y == torch.argmax(y_pred_prob,dim=1)).float().mean()
    if epoch%100==0:
        print(f'Epoch {epoch} Loss {loss.item()} Acc {accuracy}')

Epoch 0 Loss 1.127044677734375 Acc 0.30666667222976685
Epoch 100 Loss 0.04763849824666977 Acc 0.9800000190734863
Epoch 200 Loss 0.04351858049631119 Acc 0.9800000190734863
Epoch 300 Loss 0.0452808253467083 Acc 0.9800000190734863
Epoch 400 Loss 0.041653405874967575 Acc 0.9800000190734863
Epoch 500 Loss 0.04097960144281387 Acc 0.9800000190734863
Epoch 600 Loss 0.04002513363957405 Acc 0.9866666793823242
Epoch 700 Loss 0.03995898738503456 Acc 0.9866666793823242
Epoch 800 Loss 0.040136609226465225 Acc 0.9866666793823242
Epoch 900 Loss 0.03974182531237602 Acc 0.9866666793823242


最终准确率达到了98.6%